In [1]:
import os
import sys
sys.path.append('D:/ML/Kis_Api_test/1_DB_System/')

In [2]:
from communication import StockConnection, DB_Handle
import datetime
import pandas as pd
from queue import Queue

In [3]:
def date_to_string(date:datetime.date):
    conv_date = str(date)
    return conv_date[:4] + conv_date[5:7] + conv_date[-2:]

In [4]:
sc = StockConnection()
db = DB_Handle(db_name='db_for_test')
que = Queue()

good


In [5]:
kospi = pd.read_csv('../../4_SRC/current_kospi_codes.csv')
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).zfill(6))

In [6]:
kosdaq = pd.read_csv('../../4_SRC/current_kosdaq_codes.csv')
kosdaq['종목코드'] = kosdaq['종목코드'].apply(lambda x : str(x).zfill(6))

In [7]:
def result_handle(tr):
    global que
    try:
        for idx, res in enumerate(tr['output2'][::-1]):
            buf = []
            buf.append(tr['output1']['stck_shrn_iscd'])
            buf.append(res['stck_bsop_date'])
            buf.append(res['stck_clpr'])
            buf.append(res['stck_oprc'])
            buf.append(res['stck_hgpr'])
            buf.append(res['stck_lwpr'])
            buf.append(res['acml_vol'])
            buf.append(res['acml_tr_pbmn'])
            que.put(buf)
    except:     
        pass

In [12]:
for idx, context in kospi.iterrows():
    if context['회사명'] == '삼성전자':
        first_y, first_m, first_d = context['상장일'].split('-')
        first_date = datetime.date(int(first_y), int(first_m), int(first_d))
        start = datetime.date(2021, 11 ,18)#first_date
        end = start + datetime.timedelta(99)
        while end <= datetime.date.today():
            result = sc.date_price(no=context['종목코드'], 
                                   start=date_to_string(start), 
                                   end=date_to_string(end), 
                                   period='D')
            for idx, res in enumerate(result['output2'][::-1]):
                try:
                    buf = []
                    buf.append(result['output1']['stck_shrn_iscd'])
                    buf.append(res['stck_bsop_date'])
                    buf.append(res['stck_clpr'])
                    buf.append(res['stck_oprc'])
                    buf.append(res['stck_hgpr'])
                    buf.append(res['stck_lwpr'])
                    buf.append(res['acml_vol'])
                    buf.append(res['acml_tr_pbmn'])
#                     db.fetch(table_name='test', telegram=buf)
                    sql = f'INSERT INTO test {db.COLUMNS} VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
                    db.CONN.cursor().excute(sql, telegram)
                except:
                    pass
            print(f'st:{start}, end:{end}')
            start = end + datetime.timedelta(1)
            end = start + datetime.timedelta(99)
        result = sc.date_price(no=context['종목코드'], 
                                   start=date_to_string(start), 
                                   end=date_to_string(datetime.date.today()), 
                                   period='D')
        try:
            for idx, res in enumerate(result['output2'][::-1]):
                buf = []
                buf.append(result['output1']['stck_shrn_iscd'])
                buf.append(res['stck_bsop_date'])
                buf.append(res['stck_clpr'])
                buf.append(res['stck_oprc'])
                buf.append(res['stck_hgpr'])
                buf.append(res['stck_lwpr'])
                buf.append(res['acml_vol'])
                buf.append(res['acml_tr_pbmn'])
                db.fetch(table_name='test', telegram=buf)
        except:
            pass
    
    else:
        pass
db.commit()

st:2021-11-18, end:2022-02-25
st:2022-02-26, end:2022-06-05
st:2022-06-06, end:2022-09-13
st:2022-09-14, end:2022-12-22
['005930', '20221223', '58100', '58200', '58400', '57700', '9829407', '570401785680']


In [10]:
db.CUR

In [11]:
db.CONN

In [9]:
que.qsize()

0

In [1458]:
gugu = que.get()
gugu

['000880', '19920901', '5833', '5833', '5833', '5833', '78896', '4602540']

In [18]:
code = result['output1']['stck_shrn_iscd']
date = result['output2'][0]['stck_bsop_date']
clpr = result['output2'][0]['stck_clpr']
oppr = result['output2'][0]['stck_oprc']
hgpr = result['output2'][0]['stck_hgpr']
lwpr = result['output2'][0]['stck_lwpr']
volume = result['output2'][0]['acml_vol']
acpr = result['output2'][0]['acml_tr_pbmn']


In [156]:
date_to_string(date=datetime.date(2020, 7, 18))

'20200718'

In [226]:
datetime.date(2020, 7, 20) > datetime.date(2020, 7, 19)

True

In [137]:
datetime.date(2020, 7, 18) + datetime.timedelta(100)

datetime.date(2020, 10, 26)

In [142]:
result = sc.date_price(no='084010', start='20051031', end='20051031', period='D')

In [11]:
import pandas as pd

In [13]:
import pymysql

In [14]:
conn = pymysql.connect(host='localhost', user='root', 
                       password='sp_1234', db='db_for_test', charset='utf8')

In [15]:
cur = conn.cursor()

In [25]:
cur.execute(f'INSERT INTO test VALUES({code}, {date}, {clpr}, {oppr}, {hgpr}, {lwpr}, {volume}, {acpr})')


IntegrityError: (1062, "Duplicate entry '5930' for key 'test.PRIMARY'")

In [20]:
conn.commit()

TypeError: unhashable type: 'dict'